# Clinical Trial Optimization 

Je presente ici ma solution pour le probléme d'opimisation d'un trie de sujet pour un test clinique.

### Presentation du probléme

Cette compétition organisée par Ingenii portait sur l'optimisation du tri des sujets pour un essai clinique. La description complète du problème est disponible sur [la plateforme Aqora](https://aqora.io/competitions/ingenii-clinical-trial), qui héberge la compétition.

Le problème consiste à regrouper les sujets en fonction de certains paramètres lors d'un essai clinique. L'objectif est qu'à l'issue de la formation des groupes, les sujets d'un même groupe soient adaptés aux tests cliniques qu'ils devront subir.

### Formulation mathématique

Nous avons 100 sujets que nous souhaitons répartir en deux groupes $p = \{1, 2\}$ de 50 personnes chacun. Chaque sujet $i$ est caractérisé par 3 paramètres $\vec{w_i} = (w_{i1}, w_{i2}, w_{i3})$. Nous mesurons la qualité de ce partitionnement à l'aide de la grandeur suivante :

$$
d = \sum_{s=1}^{3} |\Delta\mu_s| + \rho \sum_{s=1}^{3} |\Delta\sigma_{ss}| + 2\rho \sum_{s=1}^{3} \sum_{s' = s+1}^{3} |\Delta\sigma_{ss'}|
$$

où :

$$
\Delta\mu_s = \frac{1}{n} \sum_{i=1}^{n} w_{is}(x_{i1} - x_{i2})
$$

et :

$$
\Delta\sigma_{ss'} = \frac{1}{n} \sum_{i=1}^{n} w_{is} w_{is'} (x_{i1} - x_{i2})
$$

$x_{ip}$ est une variable binaire qui vaut 1 si le sujet $i$ appartient au groupe $p$, et 0 sinon.

La grandeur $d$ est appelée la *discrepancy*. Pour un regroupement idéal, la *discrepancy* est minimale. Ce problème de regroupement se ramène donc à résoudre le problème d'optimisation suivant :

$$
\min_{x} d
$$

sous les contraintes :

$$
\sum_{i}x_{ip} = \frac{n}{2}, \quad \text{Chaque groupe contient le même nombre de sujets.}
$$

$$
x_{i1} + x_{i2} = 1, \quad \text{Un sujet ne peut appartenir qu'à un seul groupe après le regroupement.}
$$

$$
x_{12} = 0, \quad \text{Ceci pour éviter la redondance des solutions, car on obtient le même regroupement en intervertissant les groupes 1 et 2.}
$$

### Reformulation du problème

Dans sa formulation initiale, le problème semble nous orienter vers l'utilisation de l'algorithme QAOA. Cependant, cette formulation ne permet pas une implémentation directe de l'algorithme en raison de la présence de valeurs absolues dans la fonction objective. De plus, le nombre de variables impliquées est très élevé : on a effectivement $2n-1$ variables, soit 199 variables dans ce cas, ce qui nécessiterait au moins une machine de 199 qubits, ce qui n'est actuellement pas disponible.

* **Écriture sans valeurs absolues :**
$$ a = |x| \Rightarrow x \leq a \text{ et } -x \leq a$$

Nous posons alors $z_s = |\Delta\mu_s|$, $z_{ss} = |\Delta\sigma_{ss}|$, et $z_{ss'} = |\Delta\sigma_{ss'}|$. La nouvelle formulation devient :
$$
\min_{x, z} \sum_{s=1}^{3} z_s + \rho \sum_{s=1}^{3} z_{ss} + 2\rho \sum_{s=1}^{3} \sum_{s' = s+1}^{3} z_{ss'}
$$
sous les contraintes suivantes :
$$
\sum_{i} x_{ip} = \frac{n}{2}, \quad \text{chaque groupe contient le même nombre de sujets.}
$$
$$
x_{i1} + x_{i2} = 1, \quad \text{un sujet ne peut appartenir qu'à un seul groupe après le regroupement.}
$$
$$
x_{12} = 0, \quad \text{cela évite la redondance des solutions, car intervertir les groupes 1 et 2 donnerait le même regroupement.}
$$
$$
\Delta\mu_s \leq z_s \text{ et } -\Delta\mu_s \leq z_s
$$
$$
\Delta\sigma_{ss} \leq z_{ss} \text{ et } -\Delta\sigma_{ss} \leq z_{ss}
$$
$$
\Delta\sigma_{ss'} \leq z_{ss'} \text{ et } -\Delta\sigma_{ss'} \leq z_{ss'}
$$

* **Réduction du nombre de variables :**

En remplaçant la contrainte $x_{i1} + x_{i2} = 1$ dans le problème, nous réduisons de $n$ le nombre de variables. En ajoutant les variables $z$, nous obtenons finalement $n - 1 + 9$, soit $n + 8$ variables.

# Implémentation
À ce stade, le problème est bien défini pour être implémenté avec l'algorithme QAOA. La prochaine étape consiste à le transformer en un problème QUBO, une transformation bien connue et générique. Cette étape sera réalisée à l'aide d'une bibliothèque dédiée de Qiskit.

In [114]:
from qiskit_optimization import QuadraticProgram

In [115]:
n =  10 #number of subjects
N = 1 # on enléve la division par 1 dans la fonction objective
rho = 0.5

In [116]:
import csv
import numpy as np

data_list_csv = []
with open('pbc.csv', mode='r') as file:
    reader = csv.reader(file)
    # Skip the header
    next(reader)
    # Iterate over the rows and add them to the list
    for row in reader:
        # Convert strings to float for numeric columns
        data_list_csv.append([float(row[0]), float(row[1]), float(row[2])])

# je prend un arrondi à deux chiffre apres la virgule et je multiplie toute les donnée par 100 
# Pour s'assuerer que tout les coéfficlents des contraintes soient entiére. ceci est 
# necessaire pour que le probléme soit convertible en QUBO. Cette transformation ne modifie pas la combin
max_values = np.max(data_list_csv , axis=0)
data_list_frmt = data_list_csv / max_values
data_list_frmt = [[(np.around(val ,2)) for val in ligne] for ligne in data_list_frmt]



In [117]:
print(data_list_frmt[:10])

[[0.71, 0.75, 0.12], [0.62, 0.72, 0.53], [0.7, 0.89, 0.04], [0.6, 0.7, 0.44], [0.64, 0.49, 0.05], [0.64, 0.84, 0.07], [0.57, 0.71, 0.06], [0.64, 0.68, 0.34], [0.64, 0.54, 0.16], [0.67, 0.9, 0.07]]


In [118]:
mod = QuadraticProgram("Clinical trial Optimization")

for i in range(1 , n + 1):
    var_name = f"x_{i}1"
    mod.binary_var(name=var_name)

# Variables entiere   
W = []
for i in range(3):
    w_list = [row[i] for row in data_list_frmt]
    w_sum = 0
    for w in w_list:
        w_sum = w_sum + w
    W.append(w_sum)

mod.integer_var(name= "z_1" , lowerbound=0 , upperbound= int(W[0]) + 1)
mod.integer_var(name= "z_2" , lowerbound=0 , upperbound= int(W[1]) + 1)
mod.integer_var(name= "z_3" , lowerbound=0 , upperbound= int(W[2]) + 1)
#mod.continuous_var(name = "z_1" , lowerbound=0 , upperbound=int(W[0]) + 1)
#mod.continuous_var(name = "z_2" , lowerbound=0 , upperbound=int(W[1]) + 1)
#mod.continuous_var(name = "z_3" , lowerbound=0 , upperbound=int(W[2]) + 1)

W_square = []
for i in range(3):
    w_list = [row[i] for row in data_list_frmt]
    w_sum = 0
    for w in w_list:
        w_sum = w_sum + w**2
    W_square.append(w_sum)

mod.integer_var(name= "z_11" , lowerbound=0 , upperbound= int(W_square[0]) + 1)
mod.integer_var(name= "z_22" , lowerbound=0 , upperbound= int(W_square[1]) + 1)
mod.integer_var(name= "z_33" , lowerbound=0 , upperbound= int(W_square[2]) + 1)
#mod.continuous_var(name="z_11" , lowerbound=0 , upperbound=int(W_square[0]) + 1)
#mod.continuous_var(name="z_22" , lowerbound=0 , upperbound=int(W_square[1]) + 1)
#mod.continuous_var(name="z_33" , lowerbound=0 , upperbound=int(W_square[2]) + 1)


W_cross = []
for i in range(3):
    w_list_s = [row[i] for row in data_list_frmt]
    for k in range(i + 1, 3):
        w_sum = 0
        w_list_s_prime = [row[k] for row in data_list_frmt]
        for w_i, w_k in zip(w_list_s, w_list_s_prime):  
            w_sum = w_sum + w_i * w_k
        W_cross.append(w_sum)
mod.integer_var(name = "z_12" , lowerbound=0 , upperbound=int(W_cross[0] + 1))
mod.integer_var(name = "z_13" , lowerbound=0 , upperbound=int(W_cross[1] + 1))
mod.integer_var(name = "z_23" , lowerbound=0 , upperbound=int(W_cross[2] + 1))
#mod.continuous_var(name="z_12" , lowerbound=0 , upperbound=int(W_cross[0]) + 1)
#mod.continuous_var(name="z_13" , lowerbound=0 , upperbound=int(W_cross[1]) + 1)
#mod.continuous_var(name="z_23" , lowerbound=0 , upperbound=int(W_cross[2]) + 1)



<Variable: 0 <= z_23 <= 29 (integer)>

In [119]:
# definition de la fonction objective
mod.minimize(linear={"z_1": 1 ,"z_2": 1 , "z_3": 1 
                      , "z_11":0.5 , "z_22":0.5 , "z_33":0.5
                      , "z_12":1 , "z_13":1 , "z_23":1})

# Les contraintes

Contraite $ \sum_{i} x_{ip} = \frac{n}{2} \text{ avec } p = 1$

In [120]:
dict_som = {f"x_{i}1": 1 for i in range(1 , n + 1)}
mod.linear_constraint(dict_som , sense= "==" , rhs = n/2 , name = "repartition egale")

<LinearConstraint: x_101 + x_11 + x_21 + x_31 + x_41 + x_51 + x_61 + ... == 5.0 'repartition egale'>

Contrainte: $x_{12} = 0 \Rightarrow x_{11} = 1$

In [121]:
mod.linear_constraint({"x_11": 1} , sense= "==" , rhs = 1 , name= "avoid redundance")

<LinearConstraint: x_11 == 1 'avoid redundance'>

Contrainte: $\Delta\mu_s \leq z_s \text{ et } -\Delta\mu_s \leq z_s$

$$
\Delta\mu_s \leq z_s \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}}{n}x_{i1} - z_{s} - \frac{1}{n}\sum_{i = 1}^{n}w_{is} \leq 0 \\

-\Delta\mu_s \leq z_s \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}}{n}x_{i1} + z_{s} - \frac{1}{n}\sum_{i = 1}^{n}w_{is} \geq 0 
$$



In [122]:
for i in range(3):
    dict_x_s = {}
    w_list = [row[i] for row in data_list_frmt]
    dict_x_s = {f"x_{j + 1}1": (2*w_list[j])  for j in range(0 , n)}
    dict_x_s.update({f"z_{i + 1}": -1})
    #mod.linear_constraint(linear=dict_x_s , sense= "<=" , rhs= W[i])
    mod.linear_constraint(linear=dict_x_s , sense= "<=" , rhs= 1)


for i in range(3):
    dict_x_s = {}
    w_list = [row[i] for row in data_list_frmt]
    dict_x_s = {f"x_{j + 1}1": (2*w_list[j])  for j in range(0 , n)}
    dict_x_s.update({f"z_{i + 1}": 1})
    #mod.linear_constraint(linear=dict_x_s , sense= ">=" , rhs= W[i])
    mod.linear_constraint(linear=dict_x_s , sense= ">=" , rhs= 1)

Contrainte: $\Delta\sigma_{ss} \leq z_{ss} \text{ et } -\Delta\sigma_{ss} \leq z_{ss}$

$$
\Delta\sigma_{ss} \leq z_{ss} \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}^2}{n}x_{i1} - z_{ss}  \leq \frac{1}{n}\sum_{i = 1}^{n}w_{is}^2 \\

- \Delta\sigma_{ss} \leq z_{ss} \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}^2}{n}x_{i1} + z_{ss}  \geq \frac{1}{n}\sum_{i = 1}^{n}w_{is}^2 
$$

In [123]:
for i in range(3):
    dict_x_s = {}
    w_list = [row[i] for row in data_list_frmt]
    dict_x_s = {f"x_{j + 1}1": 2*(w_list[j]**2)  for j in range(0 , n)}
    dict_x_s.update({f"z_{i + 1}" + f"{i + 1}": -1})
    #mod.linear_constraint(linear=dict_x_s , sense= "<=" , rhs= W_square[i])
    mod.linear_constraint(linear=dict_x_s , sense= "<=" , rhs= 1)


for i in range(3):
    dict_x_s = {}
    w_list = [row[i] for row in data_list_frmt]
    dict_x_s = {f"x_{j + 1}1": 2*(w_list[j]**2) for j in range(0 , n)}
    dict_x_s.update({f"z_{i + 1}" + f"{i + 1}": 1})
    #mod.linear_constraint(linear=dict_x_s , sense= ">=" , rhs= W_square[i])
    mod.linear_constraint(linear=dict_x_s , sense= ">=" , rhs= 1)

In [132]:
W_square

[123.57319999999984, 132.32159999999993, 13.759899999999986]

Containte: $\Delta\sigma_{ss'} \leq z_{ss'} \text{ et } -\Delta\sigma_{ss'} \leq z_{ss'}$

$$
\Delta\sigma_{ss'} \leq z_{ss'} \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}w_{is'}}{n}x_{i1} - z_{ss'}  \leq \frac{1}{n}\sum_{i = 1}^{n}w_{is}w_{is'} \\

- \Delta\sigma_{ss'} \leq z_{ss'} \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}w_{is'}}{n}x_{i1} + z_{ss'}  \geq \frac{1}{n}\sum_{i = 1}^{n}w_{is}w_{is'} 
$$

In [125]:
for i in range(3):
    w_list_s = [row[i] for row in data_list_frmt]
    for k in range(i + 1 , 3):
        dict_x_s = {}
        w_list_s_prime = [row[k] for row in data_list_frmt]
        dict_x_s = {f"x_{j + 1}1": 2*(w_list_s[j]*w_list_s_prime[j]) for j in range(0 , n)}
        dict_x_s.update({f"z_{i + 1}" + f"{k + 1}": -1})
        #mod.linear_constraint(linear=dict_x_s , sense= "<=" , rhs= W_cross[i])
        mod.linear_constraint(linear=dict_x_s , sense= "<=" , rhs= 1)


for i in range(3):
    w_list_s = [row[i] for row in data_list_frmt]
    for k in range(i + 1 , 3):
        dict_x_s = {}
        w_list_s_prime = [row[k] for row in data_list_frmt]
        dict_x_s = {f"x_{j + 1}1": 2*(w_list_s[j]*w_list_s_prime[j]) for j in range(0 , n)}
        dict_x_s.update({f"z_{i + 1}" + f"{k + 1}": 1})
        #mod.linear_constraint(linear=dict_x_s , sense= ">=" , rhs= W_cross[i])
        mod.linear_constraint(linear=dict_x_s , sense= ">=" , rhs= 1)

In [126]:
print(mod.prettyprint())

Problem name: Clinical trial Optimization

Minimize
  z_1 + 0.5*z_11 + z_12 + z_13 + z_2 + 0.5*z_22 + z_23 + z_3 + 0.5*z_33

Subject to
  Linear constraints (20)
    x_101 + x_11 + x_21 + x_31 + x_41 + x_51 + x_61 + x_71 + x_81 + x_91
    == 5  'repartition egale'
    x_11 == 1  'avoid redundance'
    1.34*x_101 + 1.42*x_11 + 1.24*x_21 + 1.4*x_31 + 1.2*x_41 + 1.28*x_51
    + 1.28*x_61 + 1.14*x_71 + 1.28*x_81 + 1.28*x_91 - z_1 <= 1  'c2'
    1.8*x_101 + 1.5*x_11 + 1.44*x_21 + 1.78*x_31 + 1.4*x_41 + 0.98*x_51
    + 1.68*x_61 + 1.42*x_71 + 1.36*x_81 + 1.08*x_91 - z_2 <= 1  'c3'
    0.14*x_101 + 0.24*x_11 + 1.06*x_21 + 0.08*x_31 + 0.88*x_41 + 0.1*x_51
    + 0.14*x_61 + 0.12*x_71 + 0.68*x_81 + 0.32*x_91 - z_3 <= 1  'c4'
    1.34*x_101 + 1.42*x_11 + 1.24*x_21 + 1.4*x_31 + 1.2*x_41 + 1.28*x_51
    + 1.28*x_61 + 1.14*x_71 + 1.28*x_81 + 1.28*x_91 + z_1 >= 1  'c5'
    1.8*x_101 + 1.5*x_11 + 1.44*x_21 + 1.78*x_31 + 1.4*x_41 + 0.98*x_51
    + 1.68*x_61 + 1.42*x_71 + 1.36*x_81 + 1.08*x_91 + z_2 >= 

# Transformation en QUBO

La conversion d'un problème en QUBO (Quadratic Unconstrained Binary Optimization) est un processus générique sur lequel je ne me suis pas attardé en profondeur, bien qu'il soit intéressant de s'intéresser à la manière la plus optimale de réaliser cette transformation. Dans ce travail, cette conversion sera effectuée à l'aide des outils proposés par Qiskit.

In [127]:
from qiskit_optimization.converters import QuadraticProgramToQubo

cto_qubo = QuadraticProgramToQubo().convert(mod)
print(cto_qubo.prettyprint())

QiskitOptimizationError: 'Incompatible problem: Can not convert inequality constraints to equality constraint because                     float coefficients are in constraints. '

### Mapping into Ising Problem

In [107]:
hamiltonian , offset = cto_qubo.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))

KeyboardInterrupt: 

In [ ]:
from qiskit.circuit.library import QAOAAnsatz

circuit = QAOAAnsatz(cost_operator = hamiltonian , reps = 2)
circuit.measure_all()

circuit.draw('mpl')